<img src="https://geodecode.com.br/wp-content/uploads/2021/12/brasildatacube.png" align="left" style="height: 65px"/>
<img src="https://earth.bsc.es/harmonize/lib/exe/fetch.php?h=250&crop=0&tok=cfb750&media=wiki:logo.png" align="right" style="height: 65px"/>

<h1 style="color:#336699; text-align: center">BDC Lab Demostration - 2024 Harmonize Annual Meeting</h1>
<h3 style="color:#336699; text-align: center">Compute monthly health or climate data for each municipality</h3>
<hr style="border:2px solid #0077b9;">

<div style="text-align: center; font-size: 90%;">
    <a href="https://colab.research.google.com/github/Harmonize-Brazil/annual-meeting-bdclab-demo/" target = "_blank">
      <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Colab">
    </a>
    </br>
    </br>
    Luana Becker da Luz <a href="https://orcid.org/0000-0002-9916-1167"><i class="fab fa-orcid" aria-hidden="true" style="color: green"></i></a>,
    Ana Paula Dal'Asta <a href="https://orcid.org/0000-0002-1286-9067"><i class="fab fa-orcid" aria-hidden="true" style="color: green"></i></a>
    <br/>
    <br/>
    Earth Observation and Geoinformatics Division, National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: 
    <a href="mailto:luana.luz@inpe.br">luana.luz@inpe.br;</a>
    <a href="mailto:ana.dalasta@inpe.br">ana.dalasta@inpe.br;</a>
    <a href="mailto:miguel.monteiro@inpe.br">miguel.monteiro@inpe.br</a>
    <br/>
    <br/>
    <div style="width: 60%; margin: auto">
        <div style="text-align: center; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 10px;">
            This code uses RSTAC client to search health or climate collections on Harmonize STAC between start and end dates and merge all data in a single dataframe.
        </div>
    </div>
</div>

<h1 align="center">
  <img src="README_flowchart_roi.png" width="70%" style="text-align: center"/>
</h1>


In [7]:
library(sf)
library(rstac)
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# 1. Set constants
<hr style="border:1px solid #0077b9;">

In [8]:
HARMONIZE_STAC_URL <- 'https://brazildatacube.dpi.inpe.br/harmonize/dev/stac/v1/'

START_DATE <- "2020-01-01"
END_DATE <- "2020-12-31"

# 2. List Harmonize STAC collections (using RSTAC)
<hr style="border:1px solid #0077b9;">

In [9]:
all_collections <- stac(HARMONIZE_STAC_URL) %>%
  collections() %>%
  get_request()

for (collection in all_collections$collections){
  print(collection$id)
}

[1] "dengue_mortality_rate_mun_year_north-1"
[1] "total_prec_max_no_mun_epiweek-1"
[1] "max_temper_max_no_mun_epiweek-1"
[1] "max_temper_mean_ne_mun_month-1"
[1] "Phantom3Advanced_FlightHeight120m_Mosaic-1"
[1] "Phantom3Advanced_FlightHeight120m-1"
[1] "chagas_disease_cases_mun_week_north-1"
[1] "max_temper_max_no_mun_epiweek_ras-1"
[1] "max_temper_mean_no_mun_month-1"
[1] "max_temper_max_no_mun_month-1"
[1] "chagas_disease_cases_mun_month_north-1"
[1] "max_temper_max_ne_mun_epiweek-1"
[1] "dengue_cases_mun_year_north-1"
[1] "chikungunya_cases_mun_year_north-1"
[1] "zika_cases_mun_month_north-1"
[1] "dengue_cases_mun_month_north-1"
[1] "chikungunya_cases_mun_month_north-1"
[1] "dengue_cases_mun_week_north-1"
[1] "max_temper_mean_no_mun_epiweek-1"
[1] "dengue_mortality_rate_mun_month_north-1"
[1] "zika_cases_mun_year_north-1"
[1] "chagas_disease_cases_mun_year_north-1"
[1] "zika_cases_mun_week_north-1"
[1] "total_prec_max_no_mun_month-1"
[1] "dengue_mortality_rate_mun_week_north-1"
[1] 

# 3. Obtain monthly climate data for each municipality
<hr style="border:1px solid #0077b9;">

### 3.1 Set collection id (temperature, precipitation, dengue, zika, etc)

In [17]:
# COLLECTION_ID <- "dengue_cases_mun_week_north-1"
# COLLECTION_ID <- "total_prec_max_no_mun_epiweek-1"
# COLLECTION_ID <- "max_temper_mean_no_mun_epiweek-1"
COLLECTION_ID <- "max_temper_mean_no_mun_month-1"
# COLLECTION_ID <- "total_prec_max_no_mun_month-1"
# COLLECTION_ID <- "dengue_mortality_rate_mun_month_north-1"
# COLLECTION_ID <- "dengue_cases_mun_month_north-1"

### 3.2 Search climate data on Harmonize STAC (Using RSTAC)

In [18]:
stac_search <- stac(HARMONIZE_STAC_URL) %>%
  stac_search(
    collections = COLLECTION_ID,
    datetime = paste(START_DATE, END_DATE, sep = "/"),
    limit = 200
  ) %>%
  get_request()

stac_search$context

$matched
[1] 12

$returned
[1] 12

### 3.3 For each feature, retrieve geojson as dataframe

In [19]:
feature_to_geojson_csv <- function(feature) {
  datetime <- feature$properties$datetime
  geojson_href <- feature$assets$geojson$href
  
  as.data.frame(st_read(geojson_href)) %>%
    mutate(year_month = format(as.Date(datetime), "%Y-%m"))
}

dfs <- lapply(X = stac_search$features, FUN = feature_to_geojson_csv)
dfs[1:2]

Reading layer `max_temper_mean_NO_mun_month_20201201' from data source 
  `https://brazildatacube.dpi.inpe.br/harmonize/dev/data/climate_indicators/temperature_era5land/max_temper_mean_no_mun_month/shapefiles/2020-12-01/max_temper_mean_no_mun_month_20201201.geojson' 
  using driver `GeoJSON'
Simple feature collection with 21 features and 6 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -50.9967 ymin: -5.196937 xmax: -48.13749 ymax: -1.019427
Geodetic CRS:  WGS 84
Reading layer `max_temper_mean_NO_mun_month_20201101' from data source 
  `https://brazildatacube.dpi.inpe.br/harmonize/dev/data/climate_indicators/temperature_era5land/max_temper_mean_no_mun_month/shapefiles/2020-11-01/max_temper_mean_no_mun_month_20201101.geojson' 
  using driver `GeoJSON'
Simple feature collection with 21 features and 6 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -50.9967 ymin: -5.196937 xmax: -48.13749 ymax: -1.019427
Geodetic CRS:  WGS 84
Reading layer `max_t

CD_MUN,NM_MUN,SIGLA,AREA_KM2,cod,value,geometry,year_month
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<POLYGON [°]>,<chr>
1500107,Abaetetuba,PA,1610.654,NA,32.27164,POLYGON ((-48.83139 -1.5635...,2020-12
1500800,Ananindeua,PA,190.581,NA,31.91007,POLYGON ((-48.33466 -1.2398...,2020-12
1501105,Bagre,PA,4397.321,NA,32.65541,POLYGON ((-50.29255 -1.8063...,2020-12
1501204,Baião,PA,3759.834,NA,32.88814,POLYGON ((-49.48643 -2.6631...,2020-12
1501303,Barcarena,PA,1310.338,NA,31.93690,POLYGON ((-48.4695 -1.59545...,2020-12
1501402,Belém,PA,1059.466,NA,31.35751,POLYGON ((-48.35304 -1.2210...,2020-12
1501501,Benevides,PA,187.826,NA,32.35474,POLYGON ((-48.33104 -1.2569...,2020-12
1501782,Breu Branco,PA,3941.904,1,32.63459,POLYGON ((-48.92561 -3.4129...,2020-12
1502103,Cametá,PA,3081.367,NA,32.47707,POLYGON ((-49.40003 -1.9237...,2020-12


### 3.4 Rbind lists of dataframes in one big dataframe

##### If Health Data:

In [21]:
# DATA_TYPE <- "health"
DATA_TYPE <- "climate"

# COLUMN_NAME <- "dengue"
# COLUMN_NAME <- "zika"
# COLUMN_NAME <- "precipitation"
COLUMN_NAME <- "temperature"

# ------------------------------------

# rbind dataframes
rbind_dfs <- do.call(rbind, dfs)

# Select columns and rename if data is climate or health
if (DATA_TYPE == "health") {
  final_df <- rbind_dfs %>%
    select(cod, year_month, value) %>%
    rename(cd_mun = cod, !!COLUMN_NAME := value)
} else if (DATA_TYPE == "climate") {
  final_df <- rbind_dfs %>%
    select(CD_MUN, year_month, value) %>%
    rename(cd_mun = CD_MUN, !!COLUMN_NAME := value)
}

dim(final_df)
head(final_df)

[1] 252   3

,cd_mun,year_month,temperature
,<chr>,<chr>,<dbl>
1,1500107,2020-12,32.27164
2,1500800,2020-12,31.91007
3,1501105,2020-12,32.65541
4,1501204,2020-12,32.88814
5,1501303,2020-12,31.93690
6,1501402,2020-12,31.35751


### 3.5 Save data

In [9]:
FINAL_CSV_NAME <- "data_denguecases_months_2020.csv"

write.csv(final_df, FINAL_CSV_NAME, row.names = FALSE)